# Fine-tuning Llama-2 7B with QLora

### Import libraries

In [6]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

import pandas as pd
from google.colab import drive
import json
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

  error: subprocess-exited-with-error
  
  exit code: 1
  
  [51 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-312
  creating build\lib.win-amd64-cpython-312\tokenizers
  copying py_src\tokenizers\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers
  creating build\lib.win-amd64-cpython-312\tokenizers\models
  copying py_src\tokenizers\models\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\models
  creating build\lib.win-amd64-cpython-312\tokenizers\decoders
  copying py_src\tokenizers\decoders\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\decoders
  creating build\lib.win-amd64-cpython-312\tokenizers\normalizers
  copying py_src\tokenizers\normalizers\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\normalizers
  creating build\lib.win-amd64-cpython-312\tokenizers\pre_tokenizers
  copying py_src\tokenizers\pre_tokenizers\__init__.py -> build\lib.win-amd64-cpython-31

### Prepare train, test and validation data sets

In [20]:
# Get the manually annotated data aggregated using majority vote
# In this case, we take the first 50 data items as only those were annotated
file_path1 = "aggregated_labelled_data/Majority_vote.csv"
data_annotators_labels = pd.read_csv(file_path1, delimiter=';')
english_texts_annotators_labels = data_annotators_labels['english'].values

# Get the weak labels (first run the Topic_model.ipynb notebook to get these)
file_path2 = "Updated_Weak_annotations.csv"
# Skip the first 50 data items as they were labeled by annotators
data_weak_annotations = pd.read_csv(file_path2, delimiter=';', skiprows=range(1, 51))
english_texts_weak_annotations = data_weak_annotations['Document'].values
print(data_weak_annotations)

                                              Document  \
0    Clustering can be done in places licensed by t...   
1    I do see something in solar panels on roofs wh...   
2    People must be included and have confidence in...   
3    Large scale will stop growing as soon as there...   
4    should remain possible if there are good ideas...   
..                                                 ...   
427  Energy is a basic supply. I am in favour of in...   
428  The participation society means that the inhab...   
429  This is better than anything everywhere. Howev...   
430  I believe that the municipality should take th...   
431  Starting from the current situation in terms o...   

     Municipality and residents engagement in the energy sector  \
0                                                    1            
1                                                    0            
2                                                    0            
3                                  

In [21]:
def return_system():
    prompt = """
    You assign texts into topics. Answer with just the correct topics found in the text. Your answer should have maximum three topics based on the content of the text. Do not add any topics that are not listed. If no topics apply, respond with 'None of the topics are applicable'.
    """

    return prompt

def return_user(text):
    text = "Text: " + text
    prompt2 = """
    Topics:
        1. Municipality and residents engagement in the energy sector
        2. Energy storage and supplying energy in The Netherlands
        3. Wind and solar energy
        4. Market Determination Dynamics
        5. Landscapes and windmills tourism
        6. Hydrogen energy pipeline networks
    """
    return text + "\n" + prompt2

def return_prompt_format():
    format = "The response should have the following format: {'Topics': Topic1, Topic2, ...}"
    return format

In [33]:
def get_training_data_annotators_labels(number_of_samples):
    """
    Get training data from the manually annotated dataset

    Args:
        number_of_samples (int): the number of examples to use for the prompting method
    Returns:
        training_data : list of text-topics pairs

    """
    training_data = []
    sampled_data = data_annotators_labels.head(number_of_samples)
    for index, row in sampled_data.iterrows():
        # Extract the English text
        english_text = row['english']

        # Find all labels where the value is 1 to extract the topics for each text
        labels = [column for column in data_annotators_labels.columns[6:] if row[column] == 1]

        # Convert the labels into a output format for the prompt
        output = "{'Topics': " + ', '.join(labels) + "}"

        # Append the pair of English text and labels list to the pairs list
        training_data.append((english_text, output))
    return training_data


def get_training_data_weak_annotations():
    """
    Get training data from the weak annotated dataset

    Returns:
        training_data : list of text-topics pairs

    """
    # Define the topics based on the column names
    topics = [
        "Municipality and residents engagement in the energy sector",
        "Energy storage and supplying energy in The Netherlands",
        "Wind and solar energy",
        "Market Determination Dynamics",
        "Landscapes and windmills tourism",
        "Hydrogen energy pipeline networks"
    ]

    # Prepare the dictionary to store the training data
    data_training = []

    # Iterate over each row to construct the dictionary entries
    for index, row in data_weak_annotations.iterrows():
        document = row['Document']
        relevant_topics = [topic for topic, val in zip(topics, row[1:]) if val == 1]
        if relevant_topics:
            data_training.append((document, f"{{'Topics': {', '.join(relevant_topics)}}}"))
        else:
            data_training.append((document, "{'Topics': None of the topics are applicable}"))
    return data_training


def get_training_data_json_format(data_training):
    """
    Format the data into a json format used for fine-tuning the model

    Args:
        data_training : the training data with text-topic pairs
    Returns:
        data : the dataset used for fine-tuning the model

    """
    data = []
    for pair in data_training:
        prompt = return_system() + return_user(pair[0]) + return_prompt_format()
        response_message = pair[1]
        data.append({"prompt": prompt, "response": response_message})
    return data

def generate_filename(base_name, directory_name):
    """
    Generate file in a directory

    Args:
        base_name (str): Name of the file
        directory_name (str): Name of the directory
    """
    filename = f"{base_name}.jsonl"
    full_path = os.path.join(directory_name, filename)

    with open(full_path, 'w') as file:
        pass

    return full_path

def save_prints(filename, sample):
    """
    Save the data in a json format

    Args:
        filename (str): Name of the file to save the data
        sample : The dictionary of prompt-responses pairs
    """
    with open(filename, 'a') as file:
        json_sample = json.dumps(sample)  # Convert the dictionary to a JSON string
        file.write(json_sample + "\n")  # Write the JSON string to the file

def load_data_sets():
    """
    Load the train, validation and test sets
    Train/test/validation sets ratio: 80/10/10

    We have a total of 482 data items
    -> Train: 25 (from annotators' labels) + 361 (from weak labels) = 386
    -> Test: 25 (from annotators' labels) + 23 (from weak labels) = 48
    -> Validation: 49 (from weak labels) = 48

    """
    training_data_annotators_labels = get_training_data_annotators_labels(50)
    training_data_weak_labels = get_training_data_weak_annotations()

    training_data_annotators_labels_json = get_training_data_json_format(training_data_annotators_labels)
    training_data_weak_labels_json = get_training_data_json_format(training_data_weak_labels)
    print(len(training_data_weak_labels))
    # Generate and use correct filenames
    train_file_path = generate_filename("train", "content")
    validation_file_path = generate_filename("validation", "content")
    test_file_path = generate_filename("test", "content")

    # Save test data
    for i in range(25):
        save_prints(test_file_path, training_data_annotators_labels_json[i])

    for i in range(23):
        save_prints(test_file_path, training_data_weak_labels_json[i])

    # Save train data
    for i in range(25, 50):
        save_prints(train_file_path, training_data_annotators_labels_json[i])

    for i in range(23, 384):
        print(i)
        save_prints(train_file_path, training_data_weak_labels_json[i])

    # Save validation data
    for i in range(384, 432):
        save_prints(validation_file_path, training_data_weak_labels_json[i])

In [34]:
load_data_sets()

432
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
2

### Define hyperparameters

In [10]:
model_name = "NousResearch/Llama-2-7b-chat-hf"
dataset_name = "/content/train.jsonl"
new_model = "llama-2-7b-subjective-topics"
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
output_dir = "./content"
num_train_epochs = 4
fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 25
logging_steps = 25
max_seq_length = None
packing = False
device_map = {"": 0}

### Load train and test datasets from data previously extracted

In [11]:
# Load datasets
train_dataset = load_dataset('json', data_files='content/train.jsonl', split="train")
valid_dataset = load_dataset('json', data_files='content/test.jsonl', split="train")


In [12]:
# Preprocess datasets
train_dataset = train_dataset.map(lambda examples: {'text': ["<s>[INST]" + prompt + "[/INST]" + response + "</s>" + response for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)
valid_dataset = valid_dataset.map(lambda examples: {'text': ["<s>[INST]" + prompt + "[/INST]" + response + "</s>" for prompt, response in zip(examples['prompt'], examples['response'])]}, batched=True)

In [13]:
print(train_dataset)

Dataset({
    features: ['prompt', 'response', 'text'],
    num_rows: 30
})


In [17]:
# Put token from Huggingface
# https://huggingface.co/docs/hub/en/security-tokens
HF_TOKEN = 'HF_TOKEN'
# Retrieve the appropriate PyTorch data type from the torch module based on a string identifier.
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# Configure quantization parameters for the BitsAndBytes library. This setup involves specifying
# details about how the model should be quantized and in what data type it should perform computations.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,  # Boolean to decide if model weights should be loaded in 4-bit precision for the QLora algorithm
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,  # Data type for computation.
    bnb_4bit_use_double_quant=use_nested_quant,  # Boolean to enable nested quantization for additional compression.
)

# Load a pretrained model for causal language modeling from the specified model_name.
# The model is configured with the defined quantization settings and mapped to specific hardware devices.
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,  # Apply the quantization configuration to the model.
    device_map=device_map,
    token=HF_TOKEN
)

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [ ]:
model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    evaluation_strategy="steps",
    eval_steps=20  # Evaluate every 20 steps
)

In [ ]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,  # Pass validation dataset here
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

In [ ]:
trainer.train()

In [ ]:
# Save the trained model in google drive
drive.mount('/content/drive')

model_path = "/content/drive/MyDrive/" + new_model

trainer.model.save_pretrained(new_model)
os.makedirs(output_dir, exist_ok=True)
trainer.model.save_pretrained(output_dir)

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir content/runs

In [18]:
# Reload model in FP16 and merge it with QLoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

model-00001-of-00014.safetensors:   0%|          | 0.00/981M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
drive.mount('/content/drive')

model_path = "/content/drive/MyDrive/" + new_model

# Save the merged model
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

drive.mount('/content/drive')
model_path = "/content/drive/MyDrive/" + new_model

# Upload the trained model to use for prompting
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
from transformers import pipeline
prompt = "\n    You assign texts into topics. Answer with just the correct topics found in the text. Your answer should have maximum three topics based on the content of the text. Do not add any topics that are not listed. If no topics apply, respond with 'None of the topics are applicable'.\n    Text: Residents are needed. It's also a lot of fun to get involved. However, there are quick choices of principle that exclude other solutions. Very valuable on a small scale (including mienskip) and that certainly in combination with climate and reuse. Local initiatives can also lead to fragmentation while electricity must always be available, and inefficiencies.\n\n    Topics:\n        1. Municipality and residents engagement in the energy sector\n        2. Energy storage and supplying energy in The Netherlands\n        3. Wind and solar energy\n        4. Market Determination Dynamics\n        5. Landscapes and windmills tourism\n        6. Hydrogen energy pipeline networks\n    The response should have the following format: {'Topics': Topic1, Topic2, ...}"
gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=600, temperature=0.7)
result = gen(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

In [ ]:
# Import necessary libraries
from transformers import pipeline

drive.mount('/content/drive')
gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=600, temperature=0.7)

validation_prompts_file_path = "/content/validation.jsonl"

files = ["/content/drive/My Drive/finetuning_results1.txt",
         "/content/drive/My Drive/finetuning_results2.txt",
         "/content/drive/My Drive/finetuning_results3.txt",
         "/content/drive/My Drive/finetuning_results4.txt",
         "/content/drive/My Drive/finetuning_results5.txt",
         "/content/drive/My Drive/finetuning_results6.txt",
         "/content/drive/My Drive/finetuning_results7.txt",
         "/content/drive/My Drive/finetuning_results8.txt",
         "/content/drive/My Drive/finetuning_results9.txt",
         "/content/drive/My Drive/finetuning_results10.txt"]

for i in range(7):
  results_path = files[i]
  print(results_path)
  # Open the file to write results
  with open(results_path, 'w', encoding='utf-8') as results_file:
      with open(validation_prompts_file_path, 'r', encoding='utf-8') as file:
          for line in file:
              data = json.loads(line)
              prompt = data['prompt']
              # Generate the response using the model
              result = gen(f"<s>[INST] {prompt} [/INST]")
              generated_text = result[0]['generated_text']
              print(generated_text)
              result_entry = {
                  'prompt': prompt,
                  'response': generated_text
              }
              # Write the generated text to the results file
              results_file.write(json.dumps(result_entry) + "\n")
